In [1]:
###Score improvemeent from 0.55 to 0.61244
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../Titanic Prediction"))

# Any results you write to the current directory are saved as output.

['.ipynb_checkpoints', 'hravat_titanic_pred_titl.csv', 'kernel.ipynb', 'Model_Details', 'test.csv', 'train.csv']


Lets read the data and print the first few rows

In [2]:
df_main = pd.read_csv("../Titanic Prediction/train.csv")
df_main['Embarked'].fillna('S', inplace=True)
df_main['Pclass'] = df_main['Pclass'].astype(str)
df_main = df_main.loc[:,['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
#df_main = df_main.dropna(how="any", axis=0)
df_main.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


Compute the statistics for necessary columns<br>
We will not consider any categorical columns<br>
We are particularly interested in min & max values to give an idea of the number of buckets required<br> 

In [3]:
df_main_stats_summary = df_main.loc[:,['Age','Fare','SibSp','Parch']]
df_main_stats_summary.describe()

,Age,Fare,SibSp,Parch
count,714.000000,891.000000,891.000000,891.000000
mean,29.699118,32.204208,0.523008,0.381594
std,14.526497,49.693429,1.102743,0.806057
min,0.420000,0.000000,0.000000,0.000000
25%,20.125000,7.910400,0.000000,0.000000
50%,28.000000,14.454200,0.000000,0.000000
75%,38.000000,31.000000,1.000000,0.000000
max,80.000000,512.329200,8.000000,6.000000


Judging from the above values:- <br>
     1) Age :- 8 buckets<br>
     2) Fare :-  6 buckets<br>
     3) SibSP & Parch will be passed as is.

Importing the necessary packages<br>

In [4]:
import tensorflow as tf
import shutil
print(tf.__version__)

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.5.0


Lets make the input function to read data as below

In [30]:
def make_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = df['Survived'],
    batch_size = 10,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 10,
    num_threads = 1
  )

Below is the input function for predictions.<br>
Here we do not provide labels<br>

In [31]:
def make_prediction_input_fn(df, num_epochs):
  return tf.estimator.inputs.pandas_input_fn(
    x = df,
    y = None,
    batch_size = 10,
    num_epochs = num_epochs,
    shuffle = True,
    queue_capacity = 10,
    num_threads = 1  
  )

In [32]:
Pclass_list =  df_main.Pclass.unique()
Pclass_list

array(['3', '1', '2'], dtype=object)

In [33]:
gender_list =  df_main.Sex.unique()
gender_list

array(['male', 'female'], dtype=object)

In [34]:
embarked_list =  df_main.Embarked.unique()
embarked_list 

array(['S', 'C', 'Q'], dtype=object)

Below is the function for feature columns

In [35]:
def make_feature_cols():
 Pclass_categorical_column = tf.feature_column.categorical_column_with_vocabulary_list(
        key='Pclass',
        vocabulary_list=Pclass_list)
    
 Gender_categorical_column = tf.feature_column.categorical_column_with_vocabulary_list(
        key='Sex',
        vocabulary_list=gender_list)
 
 Embarked_categorical_column = tf.feature_column.categorical_column_with_vocabulary_list(
        key='Embarked',
        #vocabulary_list=embarked_list,
        vocabulary_list=embarked_list,
        default_value=0
        )


    
    
 return [  
    #Pclass_feature_column =
    tf.feature_column.embedding_column(Pclass_categorical_column,dimension=3),
    
     
    #gender_feature_column =
    tf.feature_column.embedding_column(Gender_categorical_column,dimension=2), 
    
    #age_feature_column =  
    tf.feature_column.bucketized_column(
        tf.feature_column.numeric_column('Age',dtype=tf.float32), boundaries = np.arange(0.0, 100 , 10).tolist()
        ),

    #SibSp_feature_column =  
    tf.feature_column.numeric_column('SibSp',dtype=tf.float32),
      
    #Parch_feature_column =  
    tf.feature_column.numeric_column('Parch',dtype=tf.float32), 
      
    #Fare_feature_column =  
    tf.feature_column.bucketized_column(
        tf.feature_column.numeric_column('Fare',dtype=tf.float32), boundaries = np.arange(0.0, 700 , 100).tolist()
    ),
        
    #embarked_feature_column =
    tf.feature_column.embedding_column(Embarked_categorical_column,dimension=3),
         
  ]

We now split the data into Test and training

In [36]:
np.random.seed(seed=1) #makes result reproducible
msk = np.random.rand(len(df_main)) < 0.8
traindf = df_main[msk]
evaldf = df_main[~msk]

Next we build eval specs

In [63]:
# Create estimator train and evaluate function# Creat 
def train_and_evaluate(output_dir, num_train_steps):
    
    estimator = tf.estimator.DNNClassifier(
    model_dir = output_dir,
    feature_columns=make_feature_cols(),
    hidden_units=[32,16,4],
    n_classes=2,    
    optimizer=tf.train.AdamOptimizer(
      learning_rate=0.01
    ))
    
    train_spec = tf.estimator.TrainSpec(input_fn = make_input_fn(traindf, 1000), 
                                      max_steps = num_train_steps )
    
    eval_spec = tf.estimator.EvalSpec(input_fn = make_input_fn(evaldf, 1), 
                                    steps = 1, 
                                    start_delay_secs = 60, # start evaluating after N seconds, 
                                    throttle_secs = 60 )  # evaluate every N seconds
    
    tf.estimator.train_and_evaluate(estimator, train_spec, eval_spec)
   
    return estimator

In [64]:
OUTDIR = '../Titanic Prediction/Model_Details'
tf.logging.set_verbosity(tf.logging.INFO)
shutil.rmtree(OUTDIR, ignore_errors = True)
estimator = train_and_evaluate(OUTDIR, 100)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '../Titanic Prediction/Model_Details', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002E5B8903B70>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Running training and evaluation locally (non-distributed).
INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after 60 secs (eval_spec.throttle_secs) or training is finished.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 1 into ../Titanic Prediction/Model_Details\model.ckpt.
INFO:tensorflow:loss = 6.8798904, step = 1
INFO:

In [65]:
a = estimator.predict(make_prediction_input_fn(evaldf,1))

In [66]:
df_test = pd.read_csv("../Titanic Prediction/test.csv")
df_test['Embarked'].fillna('S', inplace=True)
df_test['Pclass'] = df_test['Pclass'].astype(str)
df_test = df_test.loc[:,['Survived','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']]
#df_main = df_main.dropna(how="any", axis=0)
df_test.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,NaN,3,male,34.5,0,0,7.8292,Q
1,NaN,3,female,47.0,1,0,7.0000,S
2,NaN,2,male,62.0,0,0,9.6875,Q
3,NaN,3,male,27.0,0,0,8.6625,S
4,NaN,3,female,22.0,1,1,12.2875,S


In [67]:
test = estimator.predict(make_prediction_input_fn(df_test,1))

In [68]:
pred_list = []

for i in test:
    for a in i['classes']:
        pred_list.append(str(a)[1:][1:2])   

INFO:tensorflow:Restoring parameters from ../Titanic Prediction/Model_Details\model.ckpt-100


In [69]:
df_submit = pd.read_csv("../Titanic Prediction/test.csv")
df_submit = pd.DataFrame(df_submit['PassengerId'])
df_pred = pd.DataFrame({'Survived':pred_list})
df_submit['Survived'] = df_pred
df_submit.to_csv('hravat_titanic_pred_titl.csv',sep=',',index = False)